# Plot the posterior mass-radius countours

Here is an example of how to obtain the posteriro mass-radius countours from the posterior distribution of the EOS parameters.

After obtaining the distributions of the EOS parameters, we can map the posterior distributions of the specific EOS in mass-radius (M-R) space. This procedure helps us to understand how observational constraints affect the EOS. Each point in the EOS parameter space is uniquely correlated with a point in the EOS posterior parameter space. Then, by varying the central density, EOS points can be mapped onto the M-R plane by deriving the Tolman-Oppenheimer-Volkoff (TOV) equations.

Here is a step-by-step explanation.

First import all the package that will be used.

In [2]:
import numpy as np 
from scipy.constants import pi
from scipy.integrate import ode
from scipy.interpolate import interp1d
import random
import matplotlib.pyplot as plt
import seaborn as sns

import TOVsolver.main as main

import EOSgenerators.Strangeon_EOS as Strangeon
from TOVsolver.unit import km, Msun, MeV, fm

First, we obtain the corresponding maximum mass index for each posterior EOS parameter.

Next, to obtain the posterior MR distribution, one way is to sample the central energy density. 

Another way is to sample the MR point directly from the MR relation. This is the same concept. 

The following code is an example of sampling the MR point directly using the posterior distribution of strangeon_matter_EOS.

In [ ]:
Ms = []
Rs = []

Nq=18

array_size = 50  # Replace array_size with the desired number of random numbers 

data = np.loadtxt('equal_weighted_post.txt', delimiter=" ", skiprows=1) 
Rs = np.zeros(len(data)*array_size)
Ms = np.zeros(len(data)*array_size)
massradius=[]
for i in range(0,len(data)):
   theta = np.append(Nq, data[i][:2])  # The first two columns are epsilon/Nq, ns
   n_min = 3 * theta[2] / Nq  
   n_max = 0.16 * 8 * 3 / Nq    
   nbar_values = np.linspace(n_min, n_max, 10000)  # 10000 points between n_min and n_max  
   energy_densities, pressures = Strangeon.compute_EOS(nbar_values, theta)
   density = np.logspace(14.3, 15.6, 50)
   max_length =  len(density)

   MR = main.OutputMR('', energy_densities*MeV/fm**3, pressures*MeV/fm**3)
   
   index = len(MR[:,0])
   MFSU2R = MR[:,0]
   RFSU2R = MR[:,1]
   mr = interp1d(MFSU2R, RFSU2R)
   for j in range(0,50):
        Mpoint = max(MFSU2R)*random.random()
#A random mass Mpoint is sampled uniformly between 0 and the maximum mass in MFSU2R.
        Rpoint = mr(Mpoint) 
        Ms[i+j] =  Mpoint
        Rs[i+j] = Rpoint
#The random mass and radius values are stored in the Ms and Rs arrays.
        massradius.append([Rs[i],Ms[i]])
        print(i,Mpoint,Rpoint) 
        
plt.scatter(Rs/km, Ms/Msun, color='palevioletred')   

np.savetxt('withNICERtwo_para_Mass.txt',Ms)  
np.savetxt('withNICERtwo_para_Radius.txt',Rs)
np.savetxt('withNICERtwo_para_MR.txt',massradius) 




Then, you can use the MR points to plot the MR contour at the 99.7% confidence level.

In [ ]:
X =np.array(np.loadtxt('withNICERtwo_para_Radius.txt') )
Y=np.array(np.loadtxt('withNICERtwo_para_Mass.txt')) 
R_l = []
M_l = []
MR = list(zip(X, Y))
MR = [m for m in MR if  0.001< m[0] <16]
R_l, M_l = zip(*MR)
   

fig, ax = plt.subplots(figsize=(8, 6))
sns.kdeplot(x=R_l, y=M_l, shade=False,levels=[0.03], linestyles='--',linewidths=1.,
             cmap=None, alpha=0.9, colors=['green'], ax=ax)   
plt.scatter(R_l, M_l, color='palevioletred')  

ax.set_xlim(6, 18)
ax.set_ylim(1, 5)
ax.set_xlabel("$R~(\mathrm{km})$")
ax.set_ylabel(r"$M \ (M_{\odot}) $")
ax.tick_params(direction='in', top='on', right='on', which='both')
ax.set_title('M-R Posterior Distribution with Contour Levels')
plt.savefig("MR_posterior.pdf")
plt.show()

